# Trabalho 1 - Machine Learning II 
Prof: Carlos Padilha

#### Alunos:  

Roberto A. Coutinho  
Thais Galho


## Sistemas com Multi-classificadores ou Ensembles

#### Este trabalho visa avaliar o entendimento em relaçãao á construção de sistemas com multi-classificadores ou ensembles. Para tal, os alunos deverão fazer o seguinte:


* Implementar o algoritmo AdaBoost (nos mesmos moldes que fizemos com o algoritmo Bagging).
    – Podem escolher qualquer tipo de classificador (MLP, SVM, etc).
* Processar os dados presente no arquivo sonar.all-data.
* Realizar treinamento e teste usando validação cruzada com 10 folds.
* Avaliar os resultados em termos de acurácia, recall e precisão.

Obs: O trabalho pode ser feito em dupla e deve ser enviado por email (carlos.engcomp@gmail.com).

In [1]:
import numpy as np
import pandas as pd

# Modelos
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# K-fold CrossValidation
from sklearn.model_selection import KFold

# K-fold CrossValidation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Métricas
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [2]:
imported_data = pd.read_csv('sonar.all-data.csv', header=None)
imported_data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
# Separação entre dados e labels

labels = imported_data.iloc[:,-1]
data = imported_data.iloc[:,:-1]
len(data), len(labels)

(208, 208)

<h2>Treinamento</h2>

In [4]:

# utiliza 25% do dataset para teste
trainData, testData, trainLabels, testLabels = train_test_split(data, labels, 
                                                    train_size=0.75, 
                                                    test_size=0.25, 
                                                    stratify=labels)

print(len(trainData), len(trainLabels))
print(len(testData), len(testLabels))

(156, 156)
(52, 52)


In [5]:
n_iterations = 10



In [6]:
len(trainData), len(w), len(trainLabels)

(156, 156, 156)

In [23]:


    
    
for i in range(n_iterations):
    
    print("Interação : "+str(i))
    
    # Fit a simple decision tree first
    clf_tree = DecisionTreeClassifier(max_depth = 1, random_state = 1)
    
    kf = KFold(n_splits=10)
    
    print("Iniciando treinamento com 10 Kfolds" )
    for train_index, test_index in kf.split(trainData):
            
        print(train_index, test_index)
    
        X_train, X_test = trainData.iloc[train_index,:], trainData.iloc[test_index,:]
        print(len(X_train), len(X_test))
        
        y_train, y_test = trainLabels.iloc[train_index], trainLabels.iloc[test_index]
        print(len(y_train), len(y_test))
        
        
        n_train, n_test = len(X_train), len(X_test)

        # Initialize weights
        w = np.ones(n_train) / n_train

        pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]
        
        # Fit a classifier with the specific weights
        clf_tree.fit(X_train, y_train)

        pred_train_i = clf_tree.predict(X_train)
        print(pred_train_i)

        pred_test_i = clf_tree.predict(X_test)
        print(pred_test_i)        
        
        printCM(y_test, pred_test_i)

        
    # Indicator function
    miss = [int(x) for x in (pred_train_i != y_train)]
    print("Miss : "+str(miss))

    # Equivalent with 1/-1 to update weights
    miss2 = [x if x==1 else -1 for x in miss]

    # Error
    err_m = np.dot(w,miss) / sum(w)
    print("Error : "+str(err_m))

    # Alpha
    alpha_m = 0.2 * np.log( (1 - err_m) / float(err_m))
    print("Alpha : "+str(alpha_m))


    # New weights
    w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))
    #print("New weights : ")
    #print(w)

    print()
    
    # Add to prediction
    #pred_train = [sum(x) for x in zip(pred_train, [x * alpha_m for x in pred_train_i])]
    #pred_test = [sum(x) for x in zip(pred_test, [x * alpha_m for x in pred_test_i])]

Interação : 0
Iniciando treinamento com 10 Kfolds
(array([ 16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 155]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]))
(140, 16)
(140, 16)
['M' 'M' 'R' 'M' 'R' 'M' 'R' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'R'
 'M'

Error : 0.21985815602836858
Alpha : 0.25329863226145427
()
Interação : 2
Iniciando treinamento com 10 Kfolds
(array([ 16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 155]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]))
(140, 16)
(140, 16)
['M' 'M' 'R' 'M' '

(141, 15)
(141, 15)
['M' 'M' 'M' 'R' 'M' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M'
 'R' 'M' 'R' 'M' 'R' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'R' 'M' 'R'
 'R' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'R' 'R' 'R' 'R' 'M' 'R'
 'M' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'R' 'R' 'M' 'R' 'R' 'M' 'R' 'M' 'R' 'R'
 'R' 'M' 'M' 'R' 'M' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'R' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'R' 'M' 'M' 'R' 'R' 'R' 'M'
 'M' 'M' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'R' 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'M' 'R' 'R' 'R' 'R']
['R' 'M' 'M' 'M' 'R' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'R' 'R']
Confusion Matrix : 
[[6 1]
 [3 5]]
()
Total : 208.0
Acurácia : 73.3333333333
Misclassification rate : 0.019
True positives : 0.625
False positives : 0.143
Specificity : 0.857
Prevalence : 0.038
(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
      

['M' 'M' 'M' 'R' 'M' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M'
 'R' 'M' 'R' 'M' 'R' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'R' 'M' 'R'
 'R' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'R' 'M'
 'R' 'R' 'R' 'M' 'M' 'R' 'M' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'R'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'R' 'M' 'M' 'R' 'R'
 'R' 'M' 'M' 'M' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'R' 'M' 'M' 'M' 'R' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'R' 'R' 'R'
 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'M' 'R' 'R' 'R' 'R']
['R' 'R' 'R' 'R' 'M' 'R' 'M' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'R' 'R']
Confusion Matrix : 
[[ 4  1]
 [ 1 10]]
()
Total : 208.0
Acurácia : 87.5
Misclassification rate : 0.01
True positives : 0.909
False positives : 0.2
Specificity : 0.8
Prevalence : 0.053
(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,

(140, 16)
['M' 'M' 'R' 'M' 'R' 'M' 'R' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'R'
 'M' 'R' 'R' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'R' 'R' 'R' 'R'
 'M' 'R' 'M' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'R' 'R' 'M' 'R' 'R' 'M' 'R' 'M'
 'R' 'R' 'R' 'M' 'M' 'R' 'M' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'R'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'R' 'M' 'M' 'R' 'R'
 'R' 'M' 'M' 'M' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'R' 'M' 'M' 'M' 'R' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'R' 'R' 'R'
 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'M' 'R' 'R' 'R' 'R']
['M' 'M' 'M' 'R' 'M' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'R' 'M' 'M' 'M']
Confusion Matrix : 
[[7 2]
 [1 6]]
()
Total : 208.0
Acurácia : 81.25
Misclassification rate : 0.014
True positives : 0.857
False positives : 0.222
Specificity : 0.778
Prevalence : 0.034
(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  4

['M' 'M' 'M' 'R' 'M' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M'
 'R' 'M' 'R' 'M' 'R' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'R' 'M' 'R'
 'R' 'R' 'R' 'R' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'R' 'R' 'R' 'R' 'M' 'R'
 'M' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'R' 'R' 'M' 'R' 'R' 'M' 'R' 'M' 'R' 'R'
 'R' 'M' 'M' 'R' 'M' 'R' 'M' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'R' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'R' 'M' 'M' 'R' 'R' 'R' 'M'
 'M' 'M' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'R' 'M' 'M' 'M' 'M' 'M' 'R' 'R' 'M' 'M' 'M' 'R' 'R' 'R' 'R']
['R' 'M' 'M' 'M' 'R' 'M' 'R' 'M' 'M' 'M' 'M' 'R' 'M' 'R' 'R']
Confusion Matrix : 
[[6 1]
 [3 5]]
()
Total : 208.0
Acurácia : 73.3333333333
Misclassification rate : 0.019
True positives : 0.625
False positives : 0.143
Specificity : 0.857
Prevalence : 0.038
(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,

In [22]:
def getAccuracy(testset, predictions):
    correct = 0
    for id_test, test in enumerate(testset):
        if test == predictions[id_test]:
            correct += 1
    return (correct / float(len(testset))) * 100.0



In [18]:
from sklearn.metrics import confusion_matrix

def printCM(Y_test, predictions):
    cm = confusion_matrix(Y_test, predictions)
    print ('Confusion Matrix : ')
    print (cm)
    print()
    tn = float(cm[0][0])
    fp = float(cm[0][1])
    fn = float(cm[1][0])
    tp = float(cm[1][1])

    actual_yes = fn+tp
    actual_no = tn+fp
    predicted_yes = fp+tp
    predicted_no = tn+fn

    total = float(len(imported_data))
    print ('Total : '+ str(total))

    print ('Acurácia : ' + str(getAccuracy(Y_test, predictions)))

    misclassification_rate = round((fp+fn)/total,3) # Overall, how often is it wrong?
    print ('Misclassification rate : ' +str(misclassification_rate))

    true_positive = round(tp/actual_yes,3) # When it's actually yes, how often does it predict yes?
    print ('True positives : ' +str(true_positive))

    false_positive = round(fp/actual_no,3) # When it's actually no, how often does it predict yes?
    print ('False positives : ' +str(false_positive))

    specificity = round(tn/actual_no,3) # When it's actually no, how often does it predict no?
    print ('Specificity : ' +str(specificity))

    #precision = round(tp/predicted_yes,3) # When it predicts yes, how often is it correct?
    #print ('Precision : ' +str(precision))

    prevalence = round(actual_yes/total,3) # How often does the yes condition actually occur in our sample?
    print ('Prevalence : ' +str(prevalence))

    #f1 = round(2 * ((precision * true_positive) / (precision + true_positive)),3)
    #print ('F1 Score : ' +str(f1))

In [74]:
zip(pred_train, [x * alpha_m for x in pred_train_i])

AttributeError: 'numpy.ndarray' object has no attribute 'index'